## Steer Function Comparison
In this notebook, we examine the planners using different steer functions.

In [1]:
%load_ext autoreload
%autoreload 2
from mpb import MPB, MultipleMPB
from plot_stats import plot_planner_stats, plot_smoother_stats
from utils import latexify
from table import latex_table
from definitions import *
import matplotlib as mpl
import sys, os
mpl.rcParams['mathtext.fontset'] = 'cm'
# make sure to not use Level-3 fonts
mpl.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
from copy import deepcopy

### Polygon Environments

In [7]:
def benchmark_polygon(scenario: str, start: {str: float}, goal: {str: float}):
    time_limit_multiplier = 1
    bench = "polygon_%s" % scenario

    steer_function_times = [
        ('reeds_shepp', 15 * time_limit_multiplier),    
        ('dubins', 45 * time_limit_multiplier),    
        ('cc_reeds_shepp', 1.5 * 60 * time_limit_multiplier),    
        ('posq', 120 * time_limit_multiplier)
    ]
    
    pool = MultipleMPB()
    for steer_function, time_limit in steer_function_times:
        m = MPB()
        m["max_planning_time"] = time_limit
        m["env.start"] = start
        m["env.goal"] = goal
        m["env.type"] = "polygon"
        m["env.polygon.source"] = "polygon_mazes/%s.svg" % scenario
        m.set_steer_functions([steer_function])
        #     m.set_smoothers(['grips', 'ompl_bspline', 'ompl_shortcut', 'ompl_simplify_max'])
        #     m.set_planners(['theta_star'])
        #     m.set_planners(['sbpl_adstar'])
        pool.benchmarks.append(m)
    run_result = pool.run_parallel(bench, processes=20, limit_memory=True)

    for i, (steer_function, _) in enumerate(steer_function_times):
        bench_steer = "%s_%s" % (bench, steer_function)
        # plot trajectories
        try:
            pool.benchmarks[i].visualize_trajectories(show_legend=True, fig_width=8, fig_height=8)
            plt.savefig("%s/%s_trajectories.pdf" % (bench, bench_steer), bbox_inches='tight')
        except Exception as e:
            print("Error while visualizing trajectories:", e, file=sys.stderr)

        # plot planner stats
        try:        
            plot_planner_stats(total_filename, save_file="%s/%s_stats.pdf" % (bench, bench_steer))
        except Exception as e:
            print("Error while plotting planner statistics:", e, file=sys.stderr)

        # plot smoother stats
        try:
            plot_smoother_stats(pool.benchmarks[i].results_filename,
                                save_file="%s/%s_smoother_stats.pdf" % (bench, bench_steer),
                                separate_planners=False, num_colors=8,
                                combine_views=True, max_plots_per_line=3)
        except Exception as e:
            print("Error while plotting smoother statistics:", e, file=sys.stderr)

        # plot smoother stats (separated by planners)
        try:
            plot_smoother_stats(pool.benchmarks[i].results_filename,
                                save_file="%s/%s_smoother_stats_separated.pdf" % (bench, bench_steer),
                                separate_planners=True, num_colors=8, combine_views=True,
                                max_plots_per_line=1, fig_width=25, fig_height=6)
        except Exception as e:
            print("Error while plotting smoother statistics:", e, file=sys.stderr)

        # generate LaTeX table
        try:
            with open("%s/%s_table.txt" % (bench, bench_steer), 'w') as tf:
                tf.write(latex_table(pool.benchmarks[i].results_filename,
                                     row_label='\\textbf{Steer function: \\texttt{%s}} (\\SI{%.1f}{\\minute} time limit)'

                                     % (latexify(steer_function_names[steer_function]),
                                        time_limit/60),
                                     time_limit=time_limit))
        except Exception as e:
            print("Error while generating table:", e, file=sys.stderr)

        return run_result

In [10]:
scenarios = [
    ("parking1", {"theta": -1.58, "x": 7.72, "y": -7.72}, {"theta": 0, "x": 0, "y": -2.27}),
    ("parking2", {"theta": 0, "x": 0, "y": -2.27}, {"theta": -1.57, "x": 10.91, "y": 2.73}),
    ("parking3", {"theta": 0, "x": 15.45, "y": -2.27}, {"theta": 0, "x": 3.82, "y": 0.34}),
    ("warehouse", {"theta": -1.58, "x": -2.27, "y": 4.55}, {"theta": 1.58, "x": 63.64, "y": 2.27})
]

list(map(lambda x: benchmark_polygon(*x), scenarios))

Available memory: 12.47 GB, limiting each MPB process to 62.5% usage (7.79 GB).
Creating pool of 20 processes.
Running MPB with ID polygon_parking1_1 (log file at polygon_parking1/polygon_parking1_1.log)...
Running MPB with ID polygon_parking1_3 (log file at polygon_parking1/polygon_parking1_3.log)...
Running MPB with ID polygon_parking1_0 (log file at polygon_parking1/polygon_parking1_0.log)...
Running MPB with ID polygon_parking1_2 (log file at polygon_parking1/polygon_parking1_2.log)...


Process ForkPoolWorker-33:
Process ForkPoolWorker-28:
Process ForkPoolWorker-34:
Process ForkPoolWorker-39:
Process ForkPoolWorker-26:
Process ForkPoolWorker-36:
Process ForkPoolWorker-24:
Traceback (most recent call last):
Process ForkPoolWorker-37:
Process ForkPoolWorker-38:
Process ForkPoolWorker-22:
Process ForkPoolWorker-30:
Process ForkPoolWorker-27:
Process ForkPoolWorker-40:
Process ForkPoolWorker-29:
Process ForkPoolWorker-21:
Process ForkPoolWorker-25:
Traceback (most recent call last):
Process ForkPoolWorker-31:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-35:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-23:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process

KeyboardInterrupt: 

In [ ]:
pool.visualize_trajectories(show_legend=True)

In [ ]:
pool.benchmarks[19].visualize_trajectories(fig_width=20, fig_height=20)